In [1]:
import requests
import pandas as pd
import numpy as np
import gmaps
import json
import csv
from keys import gapikey
from matplotlib.cm import winter
from matplotlib.colors import to_hex
from area import area
import ast


gmaps.configure(api_key=gapikey)
with open('zcta/zcta.json') as a:
    geoj = json.load(a)
with open('resources/zip_targets.csv', newline='\n', encoding='utf-8-sig') as b:
    ziptgt = pd.read_csv(b)
with open('resources/dmv_pop.csv', newline='\n', encoding='utf-8-sig') as c:
    dc_pop = pd.read_csv(c)

In [2]:
# extract just population and zip from dc_pop.csv
dc_popdf = dc_pop[['GEO.id2','HC01_EST_VC01']]
# rename columns
dc_popdf = dc_popdf.rename(columns={'GEO.id2': 'zip', 'HC01_EST_VC01': 'population'}).drop(0).astype('int64')
# outer merge with target zipcode list, set index to zip and fill NaN
dc_maindf = ziptgt.merge(dc_popdf, on='zip', how='outer').set_index('zip').fillna(value=0)

cu_id = dc_maindf[ dc_maindf['city'] == 0].index
dc_maindf = dc_maindf.drop(cu_id)
cu_id = dc_maindf[ dc_maindf['population'] == 0].index 
dc_maindf = dc_maindf.drop(cu_id)

zipdict = dict(dc_maindf['population'])

dc_maindf.head()

,city,county,population
zip,,,
20001,"Washington, DC",District Of Columbia,41692.0
20002,"Washington, DC",District Of Columbia,52867.0
20003,"Washington, DC",District Of Columbia,26330.0
20004,"Washington, DC",District Of Columbia,1610.0
20005,"Washington, DC",District Of Columbia,12311.0


In [3]:
poplist = []
geoj_scrubbed = {}
areadict = {}
feature_col = []
features = geoj['features']
zip_areas = []
dc_maindf['feature'] = 0

for feature in features:
    try:
        feature_geoid = int(feature['properties']['GEOID10'])
        if feature_geoid in dc_maindf.index.values.tolist():
            feature_col.append(feature)
            poplist.append(zipdict[feature_geoid])
            dc_maindf.loc[[feature_geoid],['feature']]= str(feature)
            zarea = area(feature['geometry'])/int(1E6)
            zip_areas.append(zarea)
            areadict.update({feature_geoid:zarea})
            
            
    except ValueError:
            print(f'Value Error')
                  
geoj_scrubbed = {'type': 'FeatureCollection', 'features': feature_col}

areadf = pd.DataFrame.from_dict(areadict, orient='index', columns=['area'])

dc_maindf2 = dc_maindf.join(areadf, how='left')
dc_maindf2['density'] = dc_maindf2['population']/dc_maindf2['area']

dc_maindf2.head()

,city,county,population,feature,area,density
zip,,,,,,
20001,"Washington, DC",District Of Columbia,41692.0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",5.835524,7144.516970
20002,"Washington, DC",District Of Columbia,52867.0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",14.228399,3715.597280
20003,"Washington, DC",District Of Columbia,26330.0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",6.447784,4083.573672
20004,"Washington, DC",District Of Columbia,1610.0,"{'type': 'Feature', 'geometry': {'type': 'Mult...",0.904905,1779.192084
20005,"Washington, DC",District Of Columbia,12311.0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",1.122787,10964.678013


In [4]:
min_pop = min(dc_maindf2['population'])
max_pop = max(dc_maindf2['population'])
pop_range = max_pop - min_pop

dc_maindf2['nml_dens'] = ((dc_maindf2['population'] - min_pop)/pop_range)

dc_maindf2.head()

,city,county,population,feature,area,density,nml_dens
zip,,,,,,,
20001,"Washington, DC",District Of Columbia,41692.0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",5.835524,7144.516970,0.760585
20002,"Washington, DC",District Of Columbia,52867.0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",14.228399,3715.597280,0.964710
20003,"Washington, DC",District Of Columbia,26330.0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",6.447784,4083.573672,0.479980
20004,"Washington, DC",District Of Columbia,1610.0,"{'type': 'Feature', 'geometry': {'type': 'Mult...",0.904905,1779.192084,0.028440
20005,"Washington, DC",District Of Columbia,12311.0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",1.122787,10964.678013,0.223907


In [5]:
def calculate_color(pop):
    min_pop = min(dc_maindf2['density'])
    max_pop = max(dc_maindf2['density'])
    pop_range = max_pop - min_pop
    normalized_pop = ((pop - min_pop)/pop_range)**2
    mpl_color = winter(pop)
    gmaps_color = to_hex(mpl_color, keep_alpha=False)
    return gmaps_color

colors = []

for pop in dc_maindf2['nml_dens'].tolist():
    try:
        color = calculate_color(pop)
    except KeyError:
        color = (0, 0, 0, 0.3)
        print('key error')
    colors.append(color)
    

In [6]:
feat_list = []
for feature in dc_maindf2['feature']:
    feat_list.append(ast.literal_eval(feature))

geoj_clean = {'type': 'FeatureCollection', 'features': feat_list}

In [7]:
zcta_layer = gmaps.geojson_layer(
    geoj_clean,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig = gmaps.figure()
fig.add_layer(zcta_layer)
fig

Figure(layout=FigureLayout(height='420px'))